In [9]:
from torch_geometric.loader import DataLoader, ImbalancedSampler
from torch_geometric.transforms import FaceToEdge, OneHotDegree
import torchvision.transforms as transforms

from mantra.simplicial import SimplicialDataset
from mantra.transforms import (
    TriangulationToFaceTransform,
    OrientableToClassTransform,
    DegreeTransform,
)
from validation.validate_homology import validate_betti_numbers

import torch

In [10]:
tr = transforms.Compose(
    [
        TriangulationToFaceTransform(),
        FaceToEdge(remove_faces=False),
        DegreeTransform(),
        OrientableToClassTransform(),
        OneHotDegree(max_degree=8, cat=False),
    ]
)

dataset = SimplicialDataset(root="./data", transform=tr)


print()
print(f"Dataset: {dataset}:")
print("====================")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of features: {dataset.num_features}")
print(f"Number of classes: {dataset.num_classes}")

data = dataset[0]  # Get the first graph object.

print()
print(data)
print("=============================================================")

# Gather some statistics about the first graph.
print(f"Number of nodes: {len(data.x)}")
print(f"Number of edges: {data.num_edges}")
print(f"Average node degree: {data.num_edges / len(data.x):.2f}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self-loops: {data.has_self_loops()}")
print(f"Is undirected: {data.is_undirected()}")

print("=============================================================")
print(f"Number of orientable Manifolds: {sum(dataset.orientable)}")
print(
    f"Number of non-orientable Manifolds: {len(dataset) - sum(dataset.orientable)}"
)
print(
    f"Percentage: {sum(dataset.orientable) / len(dataset):.2f}, {(len(dataset) - sum(dataset.orientable)) / len(dataset):.2f}"
)


Dataset: SimplicialDataset(712):
Number of graphs: 712
Number of features: 9
Number of classes: 2

Data(dimension=[1], n_vertices=[1], torsion_coefficients=[3], betti_numbers=[3], orientable=[1], genus=[1], name='S^2', face=[3, 4], edge_index=[2, 12], x=[4, 9], y=[1])
Number of nodes: 4
Number of edges: 12
Average node degree: 3.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True
Number of orientable Manifolds: 193
Number of non-orientable Manifolds: 519
Percentage: 0.27, 0.73


In [11]:
data.betti_numbers

[1, 0, 1]

In [12]:
dataset = dataset.shuffle()

train_dataset = dataset[:-150]
test_dataset = dataset[-150:]

print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of test graphs: {len(test_dataset)}")

Number of training graphs: 562
Number of test graphs: 150


In [13]:
train_loader = DataLoader(
    train_dataset, batch_size=10
)  # ,sampler=ImbalancedSampler(train_dataset))
test_loader = DataLoader(test_dataset, batch_size=10)


for batch in train_loader:
    break

batch.y

tensor([1, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [14]:
from operator import concat
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TAGConv, TransformerConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = TAGConv(dataset.num_node_features, hidden_channels)
        self.conv2 = TAGConv(hidden_channels, hidden_channels)
        self.conv3 = TAGConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 3)

    def forward(self, batch):
        # 1. Obtain node embeddings
        x = self.conv1(batch.x, batch.edge_index)
        x = x.relu()
        x = self.conv2(x, batch.edge_index)
        x = x.relu()
        x = self.conv3(x, batch.edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch.batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x


model = GCN(hidden_channels=64)
print(model(batch))

tensor([[-0.0592, -0.0860,  0.0107],
        [-0.0745, -0.0838, -0.0963],
        [-0.1234, -0.1174,  0.0508],
        [-0.2945, -0.0624,  0.3497],
        [-0.3387, -0.0551, -0.0335],
        [-0.1517, -0.1775, -0.2044],
        [-0.1485, -0.0703, -0.0104],
        [-0.1371, -0.1050, -0.0641],
        [-0.0389, -0.2190,  0.0169],
        [ 0.0023, -0.2771, -0.1588]], grad_fn=<AddmmBackward0>)


In [15]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()


def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data)  # Perform a single forward pass.
        loss = criterion(
            out, torch.tensor(data.betti_numbers, dtype=torch.float)
        )  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(loader):
    model.eval()

    losses = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data)

        losses += criterion(
            out, torch.tensor(data.betti_numbers, dtype=torch.float)
        )
    return losses


for epoch in range(1, 40):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(
        f"Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}"
    )

Epoch: 001, Train Acc: 6.5337, Test Acc: 2.0014
Epoch: 002, Train Acc: 6.1711, Test Acc: 1.7780
Epoch: 003, Train Acc: 5.5158, Test Acc: 1.5298
Epoch: 004, Train Acc: 5.0714, Test Acc: 1.4182
Epoch: 005, Train Acc: 7.9687, Test Acc: 2.0118
Epoch: 006, Train Acc: 5.9958, Test Acc: 1.6171
Epoch: 007, Train Acc: 4.3281, Test Acc: 1.3416
Epoch: 008, Train Acc: 4.8488, Test Acc: 1.4623
Epoch: 009, Train Acc: 4.3541, Test Acc: 1.3209
Epoch: 010, Train Acc: 4.5998, Test Acc: 1.2991
Epoch: 011, Train Acc: 4.9841, Test Acc: 1.4021
Epoch: 012, Train Acc: 4.2027, Test Acc: 1.3458
Epoch: 013, Train Acc: 4.1259, Test Acc: 1.2443
Epoch: 014, Train Acc: 4.9470, Test Acc: 1.5142
Epoch: 015, Train Acc: 4.3742, Test Acc: 1.3832
Epoch: 016, Train Acc: 4.3983, Test Acc: 1.3189
Epoch: 017, Train Acc: 4.8632, Test Acc: 1.4811
Epoch: 018, Train Acc: 4.1164, Test Acc: 1.2966
Epoch: 019, Train Acc: 4.2895, Test Acc: 1.3772
Epoch: 020, Train Acc: 4.3439, Test Acc: 1.4044
Epoch: 021, Train Acc: 4.5521, Test Acc:

In [25]:
model.eval()
losses = 0
y_hat = []
y = []
for data in test_loader:  # Iterate in batches over the training/test dataset.
    out = model(data)
    y_hat.append(out.round().long())
    y.append(torch.tensor(data.betti_numbers))

y_hat = torch.vstack(y_hat)
y = torch.vstack(y)


incorrect = torch.count_nonzero(y - y_hat != 0).item()
correct = torch.count_nonzero(y - y_hat == 0).item()

print("predicted incorrect", correct)
print("predicted incorrect", incorrect)
print("percentage correct", correct / (correct + incorrect))

predicted incorrect 391
predicted incorrect 59
percentage correct 0.8688888888888889
